In [2]:
#!/usr/bin/env python
# Read data from an opendap server
import netCDF4
# specify an url, the JARKUS dataset in this case
url = 'http://thredds.ucar.edu/thredds/dodsC/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170412_2000.gini'
# for local windows files, note that '\t' defaults to the tab character in python, so use prefix r to indicate that it is a raw string.
# create a dataset object
dataset = netCDF4.Dataset(url)
 
# lookup a variable
dataset.variables

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
              float64 time(time)
                  long_name: time since base date
                  _CoordinateAxisType: Time
                  units: msecs since 1970-01-01T00:00:00Z
              unlimited dimensions: 
              current shape = (1,)
              filling off), ('x', <class 'netCDF4._netCDF4.Variable'>
              float64 x(x)
                  long_name: projection x coordinate
                  units: km
                  _CoordinateAxisType: GeoX
              unlimited dimensions: 
              current shape = (1536,)
              filling off), ('y', <class 'netCDF4._netCDF4.Variable'>
              float64 y(y)
                  long_name: projection y coordinate
                  units: km
                  _CoordinateAxisType: GeoY
              unlimited dimensions: 
              current shape = (1008,)
              filling off), ('Stereographic',
              <class 'netCDF4._netCDF4.Variab

In [4]:
#SFC_T - Surface Temperature
print(dataset.variables["S FC_T"][:])

[[[  0   0   0 ...,   0   0   0]
  [  0   0   0 ...,   0   0   0]
  [  0   0   0 ...,   0   0   0]
  ..., 
  [ 70  70  70 ...,   0   0   0]
  [ 70  70 -49 ...,   0   0   0]
  [ 70 -50 -49 ...,   0   0   0]]]
